<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/99.Machine_Learning/11_decision_trees_and_random_forests_Bank_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Decision Trees y Random Forests

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import seaborn as sns



En este notebook vamos a aprender a utilizar Árboles de Decisión en Scikit-learn.
Como veremos, los árboles de decisión son extremadamente intiutivos. En el fondo codifican una serie de decisiones de la misma manera que lo haría una sentencia condicional *if-else*. La diferencia es que, en lugar de programar estas decisiones manualmente, el árbol aprende a generarlas automáticamente en función de los datos de entrenamiento.
Por ejemplo, si queremos utilizar una guía para identificar animales, podríamos preguntar las siguientes preguntas:

- Este animal, mide más de un metro de longitud?
    + *Más de 1m long*. ¿Tiene cuernos?
        - *Sí*: ¿Miden sus cuernos más de 10cm de longitud?
        - *No*: ¿El animal tiene pelo o escamas?
    + *Menos de 1m *: ¿El animal tiene 2 o 4 patas?
        - *Dos patas*: ¿Tiene alas?
        - *Cuatro patas*: ¿Tiene cola de más de 12 cm de largo?
  
Y así podría seguir... La esencia de un árbol de decisión es ir realizando preguntas que permitan hacer particiones binarias del dataset.

Una de las principales ventajas de los árboles es que necesitan muy poco pre-procesado (p.ej. son invariantes al escalado de las variables) y permiten trabajar con diferentes tipos de variables (contínuas, discretas). 



## Decision Trees: Regresión



Vamos a empezar con un ejemplo de árboles de Regresión. Para ello crearemos un árbol de decisión que intentará ajustar los datos sinténticos que vamos a crear a continuación:

In [ ]:
def make_dataset(n_samples=100):
    rnd = np.random.RandomState(42)
    x = np.linspace(-3, 3, n_samples)
    y_no_noise = np.sin(4 * x) + x
    y = y_no_noise + rnd.normal(size=len(x))
    return x, y

In [ ]:
x, y = make_dataset()
X = x.reshape(-1, 1)

plt.xlabel('Feature X')
plt.ylabel('Target y')
plt.scatter(X, y);

In [ ]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(max_depth=5)
reg.fit(X, y)

X_fit = np.linspace(-3, 3, 1000).reshape((-1, 1))
y_fit_1 = reg.predict(X_fit)

plt.plot(X_fit.ravel(), y_fit_1, color='navy', label="prediction")
plt.plot(X.ravel(), y, '.k', label="training data")
plt.legend(loc="best");



##### Como vemos, el modelo genera zonas con líneas planas muy anchas (bias) y zonas con cambios verticales muy bruscos ajustándose a uno o muy pocos puntos (overfitting).

Esto es uno de los problemas típicos de los árboles que se puede controlar con la profundidad del mismo. 

## Ejercicio

- Probar varios parámetros de profundidad y observar como aumentan el bias y la varianza del modelo.



# Decision Tree: Clasificación

Los problemas de clasificación también se pueden resolver árboles de decisión.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importo el CSV creado previamente

df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/UADE/Diplomatura Ciencia de Datos/Material de Clases/MESES_201402_201404.csv', dtype={"numero_de_cliente": int, "foto_mes": int, "cliente_sucursal": "string"})


In [ ]:
df2.head()

In [ ]:
# Elimino la primer columna
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]

In [ ]:
df2.describe(include='all')  

In [ ]:
df2.info()

In [ ]:
df2.dtypes

In [ ]:
X = df2.drop(['target' , 'target_bin'] , axis='columns')
y = df2.target_bin

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
X = X.select_dtypes(include=numerics)
X = X.fillna(0)

In [ ]:
X.describe()

In [ ]:
s = df2.target_bin
counts = s.value_counts()
percent = s.value_counts(normalize=True)
percent100 = s.value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})

In [ ]:
#Entreno un Arbol de Decición
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# Train model
clf = DecisionTreeClassifier().fit(X_train, y_train)
#clf = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
#clf = DecisionTreeClassifier(max_depth=5, class_weight='balanced').fit(X_train, y_train)
 
# Predict on training set and test set
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [ ]:
# Accuracy 
from sklearn.metrics import accuracy_score
print('Accuracy train:',accuracy_score(y_train, y_train_pred))
print('Accuracy test:',accuracy_score(y_test, y_test_pred))

In [ ]:
# veo la matriz de confusión en train
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)

In [ ]:
# veo la matriz de confusión en test
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred)

In [ ]:
#Ahora puedo obtener Recall y Precision
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print('Accuracy')
print('Train:',accuracy_score(y_train, y_train_pred))
print('Test:',accuracy_score(y_test, y_test_pred))
print(' ')
print('Recall')
print('Train:',recall_score(y_train, y_train_pred))
print('Test:',recall_score(y_test, y_test_pred))
print(' ')
print('Precision')
print('Train:',precision_score(y_train, y_train_pred))
print('Test:',precision_score(y_test, y_test_pred))


In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_test_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))

In [ ]:
    print()
    print(format('How to plot a ROC Curve in Python','*^82'))

    import warnings
    warnings.filterwarnings("ignore")

    # load libraries
    from sklearn.datasets import make_classification
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve, roc_auc_score
    from sklearn.model_selection import train_test_split
    import matplotlib.pyplot as plt

    # Create feature matrix and target vector
    #X, y = make_classification(n_samples=10000, n_features=100, n_classes=2)

    # Split into training and test sets
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    # Create classifier
    clf1 = DecisionTreeClassifier()
    clf2 = DecisionTreeClassifier(max_depth=5)
    clf3 = LogisticRegression()

    # Train model
    clf1.fit(X_train, y_train)
    clf2.fit(X_train, y_train)
    clf3.fit(X_train, y_train)

    # Get predicted probabilities
    y_score1 = clf1.predict_proba(X_test)[:,1]
    y_score2 = clf2.predict_proba(X_test)[:,1]
    y_score3 = clf3.predict_proba(X_test)[:,1]

    # Plot Receiving Operating Characteristic Curve
    # Create true and false positive rates
    false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_score1)
    false_positive_rate2, true_positive_rate2, threshold2 = roc_curve(y_test, y_score2)
    false_positive_rate3, true_positive_rate3, threshold3 = roc_curve(y_test, y_score3)
    print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))
    print('roc_auc_score for DecisionTree_5: ', roc_auc_score(y_test, y_score2))
    print('roc_auc_score for Logistic Regression: ', roc_auc_score(y_test, y_score3))

    # Plot ROC curves
    plt.subplots(1, figsize=(10,10))
    plt.title('Receiver Operating Characteristic - DecisionTree')
    plt.plot(false_positive_rate1, true_positive_rate1)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

    plt.subplots(1, figsize=(10,10))
    plt.title('Receiver Operating Characteristic - DecisionTree Levels 5')
    plt.plot(false_positive_rate2, true_positive_rate2)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

    plt.subplots(1, figsize=(10,10))
    plt.title('Receiver Operating Characteristic - Logistic regression')
    plt.plot(false_positive_rate3, true_positive_rate3)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
## import dependencies
from sklearn import tree #For our Decision Tree
import pandas as pd # For our DataFrame
import pydotplus # To create our Decision Tree Graph
from IPython.display import Image  # To Display a image of our graph


# The decision tree classifier.
#clf2 = tree.DecisionTreeClassifier(max_depth=5)
clf2 = tree.DecisionTreeClassifier()
# Training the Decision Tree
clf_train = clf2.fit(X_train, y_train)


# Export/Print a decision tree in DOT format.
#print(tree.export_graphviz(clf_train, None))

#Create Dot Data
dot_data = tree.export_graphviz(clf_train, out_file=None, feature_names=list(X_train.columns.values), 
                                class_names=['0', '1'], rounded=True, filled=True) #Gini decides which attribute/feature should be placed at the root node, which features will act as internal nodes or leaf nodes
#Create Graph from DOT data
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())

In [ ]:
#Entreno un Arbol de Decición con Cross Validation
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=0)

cross_val_score(clf, X, y, cv=10)

In [ ]:
# Veo los parametros
clf.predict



 ## Ejercicio

Entrenar el modelo árboles más profundos.

In [ ]:
# Respuesta aqui



# Random Forests



Random Forests son ensamblajes de muchos árboles entrenados con diferentes subconjuntos de datos y diferentes subconjuntos de variables del dataset de entrenamiento. 
Esto permite que los árboles del bosque sean diferentes entre ellos. De hecho, cada árbol se especializa en un aspecto diferente del dataset (sobreajusta al subconjunto de datos con los que se ha entrenado ese árbol), pero como la predicción es un promedio de las predicciones de todos los árboles, al final el bosque es capaz de reducir la varianza (overfitting).



## Seleccionando los parámetros de un bosque utilizando Cross-Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rf = RandomForestClassifier(n_estimators=50) #200
parameters = {'max_features':['sqrt', 'log2', 10],
              'max_depth':[5, 7, 9]}

clf_grid = GridSearchCV(rf, parameters, n_jobs=-1)
clf_grid.fit(X_train, y_train)

In [ ]:
clf_grid.score(X_train, y_train)

In [ ]:
# Predict on training set and test set
y_train_pred = clf_grid.predict(X_train)
y_test_pred = clf_grid.predict(X_test)

In [ ]:
print('Accuracy')
print('Train:',accuracy_score(y_train, y_train_pred))
print('Test:',accuracy_score(y_test, y_test_pred))
print(' ')
print('Recall')
print('Train:',recall_score(y_train, y_train_pred))
print('Test:',recall_score(y_test, y_test_pred))
print(' ')
print('Precision')
print('Train:',precision_score(y_train, y_train_pred))
print('Test:',precision_score(y_test, y_test_pred))

In [ ]:
clf_grid.score(X_test, y_test)

In [ ]:
clf_grid.best_params_



## Gradient Boosting



Otro método de ensemble es *Boosting*. Aquí, en lugar de construir 200 estimadores en paralelo (como hacíamos en el bosque anterior), construimos una cadena de 200 estimadores que iterativamente van refinando los resultados del estimador anterior. La idea de boosting es que, utilizar modelos muy simples y rápidos secuencialmente nos permite obtener un error menor que el que pueden obtener los distintos estimadores individualmente.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=.2)
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))



## Ejercicio: Gradient Boosting con cross-validation



- Utilizad una búsqueda en grid para optimizar los parámetros `learning_rate` (0.01-1) y `max_depth` (1-9) de un Gradient Boosted Trees en el dataset de Bank

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier





In [ ]:
# Respuesta aqui